# The Notebook Calculates A Simulation Model
<font color="red">MUST RUN DOWNLOAD CMS Files notebook first</font>

This notebook will use the Medicare Part B files to calculate a simulation of what the Medicare Physician Fee Schedule proposal. Once this notebook is finished, it will save the model using the medicare_payment_model_saved variable as the file name.

The calculation take a fair amount of time. It may take the notebook <font color="red">over 10 minutes</font> to complete.

In [1]:
medicare_saved = "medicare_saved.pkl"
medicare_agg_saved = "medicare_agg_saved.pkl"

medicare_payment_model_saved = "medicare_payment_model.pkl"

In [2]:
import numpy as np
import pandas as pd

import multiprocessing
import pandas as pd
import numpy as np

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))

These are the two formulas that calculate the model. The `calculateRates` function contains the calculations that depend on the CPT code. The `calculateProposedRates` are the calculations that are based on the per NPI. 

Currently, the model only calculates for Primary Care (only Family Physicians currently). There are some assumptions made:
 * Time saved are only calculated for Level 3-5 visits & 100% is converted to saved dollars
 * To estimate the number of prolonged visits, use 70% of 99205 and 50% of 99215
 * Family Physician average hourly income is $93/hr **other specialties need incomes**

In [3]:
# Specialty:[primary care gcode, specialty gcode, salary/minute, placeholder] 0 = zero, 1 = 100%
provider_dict = {
    'Addiction Medicine':[0,0,0,0],'All Other Suppliers':[0,0,0,0],'Allergy/ Immunology':[0,1,0,0],
    'Ambulance Service Provider':[0,0,0,0],'Ambulatory Surgical Center':[0,0,0,0],'Anesthesiology':[0,0,0,0],
    'Anesthesiology Assistant':[0,0,0,0],'Audiologist':[0,0,0,0],'Cardiac Surgery':[0,0,0,0],
    'Cardiovascular Disease (Cardiology)':[0,1,0,0],'Centralized Flu':[0,0,0,0],'Certified Clinical Nurse Specialist':[0,0,0,0],
    'Certified Nurse Midwife':[0,0,0,0],'Certified Registered Nurse Anesthetist (CRNA)':[0,0,0,0],'Chiropractic':[0,0,0,0],
    'Clinic or Group Practice':[0,0,0,0],'Clinical Cardiatric Electrophysiology':[0,0,0,0],'Clinical Laboratory':[0,0,0,0],
    'Colorectal Surgery (Proctology)':[0,0,0,0],'Critical Care (Intensivists)':[0,0,0,0],'Dentist':[0,0,0,0],'Dermatology':[0,0,0,0],
    'Diagnostic Radiology':[0,0,0,0],'Emergency Medicine':[0,0,0,0],'Endocrinology':[0,1,0,0],'Family Practice':[1,0,1.55,0],
    'Gastroenterology':[0,0,0,0],'General Practice':[1,0,0,0],'General Surgery':[0,0,0,0],'Geriatric Medicine':[0,0,0,0],
    'Geriatric Psychiatry':[0,0,0,0],'Gynecological Oncology':[0,0,0,0],'Hand Surgery':[0,0,0,0],'Hematology':[0,0,0,0],
    'Hematology-Oncology':[0,1,0,0],'Hospice and Palliative Care':[0,0,0,0],'Hospitalist':[0,0,0,0],'Independent Diagnostic Testing Facility (IDTF)':[0,0,0,0],
    'Infectious Disease':[0,0,0,0],'Internal Medicine':[1,0,0,0],'Interventional Cardiology':[0,0,0,0],'Interventional Pain Management':[0,1,0,0],
    'Interventional Radiology':[0,0,0,0],'Licensed Clinical Social Worker':[0,0,0,0],'Mammography Center':[0,0,0,0],'Mass Immunizer Roster Biller':[0,0,0,0],
    'Maxillofacial Surgery':[0,0,0,0],'Medical Oncology':[0,0,0,0],'Nephrology':[0,0,0,0],'Neurology':[0,1,0,0],'Neuropsychiatry':[0,0,0,0],
    'Neurosurgery':[0,0,0,0],'Nuclear Medicine':[0,0,0,0],'Nurse Practitioner':[0,0,0,0],'Obstetrics & Gynecology':[0,1,0,0],
    'Occupational Therapist in Private Practice':[0,0,0,0],'Ophthalmology':[0,0,0,0],'Optometry':[0,0,0,0],'Oral Surgery (Dentists only)':[0,0,0,0],
    'Orthopedic Surgery':[0,0,0,0],'Osteopathic Manipulative Medicine':[0,0,0,0],'Otolaryngology':[0,1,0,0],'Pain Management':[0,0,0,0],
    'Pathology':[0,0,0,0],'Pediatric Medicine':[1,0,0,0],'Peripheral Vascular Disease':[0,0,0,0],'Physical Medicine and Rehabilitation':[0,0,0,0],
    'Physical Therapist in Private Practice':[0,0,0,0],'Physician Assistant':[0,0,0,0],'Plastic and Reconstructive Surgery':[0,0,0,0],
    'Podiatry':[0,0,0,0],'Portable X-Ray Supplier':[0,0,0,0],'Preventive Medicine':[0,0,0,0],'Psychiatry':[0,0,0,0],'Psychologist, Clinical':[0,0,0,0],
    'Public Health or Welfare Agency':[0,0,0,0],'Pulmonary Disease':[0,0,0,0],'Radiation Oncology':[0,0,0,0],'Radiation Therapy Center':[0,0,0,0],
    'Registered Dietitian or Nutrition Professional':[0,0,0,0],'Rheumatology':[0,1,0,0],'Sleep Medicine':[0,0,0,0],'Slide Preparation Facility':[0,0,0,0],
    'Speech Language Pathologist':[0,0,0,0],'Sports Medicine':[0,0,0,0],'Surgical Oncology':[0,0,0,0],'Thoracic Surgery':[0,0,0,0],
    'Undefined Physician type':[0,0,0,0],'Unknown Supplier/Provider Specialty':[0,0,0,0],'Urology':[0,1,0,0],'Vascular Surgery':[0,0,0,0]
    }

# [Current Payment Rate, Proposed Payment Rate, Fraction of encounters that complexity Gcodes applies
#        , Fraction of encounters where time is saved, Fraction of encounters that might be > 16 min ]
rowRates = {
        '99201':[45,44,0,0,0],'99202':[76,135,0,0,0],
        '99203':[110,13,0,1,0],'99204':[167,135,0,1,0],'99205':[211,135,0,1,0.7],
        '99211':[22,24,1,0,0],'99212':[45,93,1,0,0],
        '99213':[74,93,1,1,0],'99214':[109,93,1,1,0],'99215':[148,93,1,1,0.5]
    }

def calculateRates(row, p_dict=provider_dict, rowRates=rowRates):
    pc_rate = 5 #Primary Care G-Code Rate
    spec_rate = 9 #Specialty G-Code Rate
    pType = p_dict.get(row['PROVIDER_TYPE'])
    
    rR = rowRates.get(row['HCPCS_CODE'])

    row['CURRENT_RATE'] = row['LINE_SRVC_CNT'] * rR[0] # Current Payment
    row['PROPOSED_RATE'] = row['LINE_SRVC_CNT'] * rR[1] # Proposed Payment
    
    # Calculate Primary Care Complexity Gcode
    row['PRIMARYCARE_GCODE'] = row['LINE_SRVC_CNT'] * rR[2] * pc_rate * pType[0] # Primary Care G-Code Payment
    row['PRIMARYCARE_GCODE_PLUS5'] = row['LINE_SRVC_CNT'] * rR[2] * (pc_rate + 5) * pType[0] # Primary Care G-Code Payment
    row['PRIMARYCARE_GCODE_PLUS8'] = row['LINE_SRVC_CNT'] * rR[2] * (pc_rate + 8) * pType[0] # Primary Care G-Code Payment
    row['PRIMARYCARE_GCODE_PLUS10'] = row['LINE_SRVC_CNT'] * rR[2] * (pc_rate + 10) * pType[0] # Primary Care G-Code Payment
    
    # Calculate Specialty Complexity Gcode
    row['SPECIALTY_GCODE'] = row['LINE_SRVC_CNT'] * rR[2] * spec_rate * pType[1]
    
    row['MIN_SAVED'] = 1.6 * rR[3] * row['LINE_SRVC_CNT']
    
    #Calculate Prolonged Visits
    row['PROLONGED_VISIT_GCODE'] = row['LINE_SRVC_CNT'] * rR[4] * 67
    return row

def calculateProposedRates(row, p_dict=provider_dict):
    pType = p_dict.get(row['PROVIDER_TYPE'])
    
    # Add together the proposed base rate and the primary care complexity g-code rate
    row['PROPOSED_RATE_WITH_GCODE'] = row['PROPOSED_RATE'] + row['PRIMARYCARE_GCODE']
    row['PROPOSED_RATE_WITH_GCODE_PLUS5'] = row['PROPOSED_RATE'] + row['PRIMARYCARE_GCODE_PLUS5']
    row['PROPOSED_RATE_WITH_GCODE_PLUS8'] = row['PROPOSED_RATE'] + row['PRIMARYCARE_GCODE_PLUS8']
    row['PROPOSED_RATE_WITH_GCODE_PLUS10'] = row['PROPOSED_RATE'] + row['PRIMARYCARE_GCODE_PLUS10']

    
    # Calculate the difference between the proposed new rate and old current rate WITHOUT primary care g-code
    row['DIFFERENCE'] = row['PROPOSED_RATE'] - row['CURRENT_RATE']
    # Calculate the difference between the proposed new rate and old current rate WITH primary care g-code
    row['DIFFERENCE_WITH_GCODE'] = row['PROPOSED_RATE_WITH_GCODE'] - row['CURRENT_RATE']
    row['DIFFERENCE_WITH_GCODE_PLUS5'] = row['PROPOSED_RATE_WITH_GCODE_PLUS5'] - row['CURRENT_RATE']
    row['DIFFERENCE_WITH_GCODE_PLUS8'] = row['PROPOSED_RATE_WITH_GCODE_PLUS8'] - row['CURRENT_RATE']
    row['DIFFERENCE_WITH_GCODE_PLUS10'] = row['PROPOSED_RATE_WITH_GCODE_PLUS10'] - row['CURRENT_RATE']


    # Create a new column with the payment difference normalize to number of unique beneficaries per NPI
    row['DIFFERENCE_WITH_GCODE_PER_BENE'] = row['DIFFERENCE_WITH_GCODE'] / row['TOTAL_UNIQUE_BENES']
    row['PROPOSED_RATE_WITH_GCODE_PER_BENE'] = row['PROPOSED_RATE_WITH_GCODE'] / row['TOTAL_UNIQUE_BENES']
    row['DIFFERENCE_WITH_GCODE_PLUS5_PER_BENE'] = row['DIFFERENCE_WITH_GCODE_PLUS5'] / row['TOTAL_UNIQUE_BENES']
    row['DIFFERENCE_WITH_GCODE_PLUS8_PER_BENE'] = row['DIFFERENCE_WITH_GCODE_PLUS8'] / row['TOTAL_UNIQUE_BENES']
    row['DIFFERENCE_WITH_GCODE_PLUS10_PER_BENE'] = row['DIFFERENCE_WITH_GCODE_PLUS10'] / row['TOTAL_UNIQUE_BENES']
    
    #Calculate $ equivalent to time saving.
    row['DOLLARS_SAVED'] = row['MIN_SAVED']* pType[2]

    return row

In [4]:
df_medicare = pd.read_pickle(medicare_saved)
df_medicare.head()

,NPI,PLACE_OF_SERVICE,HCPCS_CODE,LINE_SRVC_CNT,PROVIDER_TYPE
0,1003000126,F,99217,57.0,Internal Medicine
1,1003000126,F,99219,38.0,Internal Medicine
2,1003000126,F,99220,23.0,Internal Medicine
3,1003000126,F,99221,20.0,Internal Medicine
4,1003000126,F,99222,96.0,Internal Medicine


Currently, the notebook focuses only on Family Practice and the E&M codes in the list

In [14]:
df_med_filtered = df_medicare[df_medicare.HCPCS_CODE.isin(['99201', '99202','99203','99204','99205','99211', '99212','99213','99214','99215'])]
df_med_filtered.head()

,NPI,PLACE_OF_SERVICE,HCPCS_CODE,LINE_SRVC_CNT,PROVIDER_TYPE
27,1003000142,O,99204,91.0,Anesthesiology
28,1003000142,O,99205,11.0,Anesthesiology
29,1003000142,O,99213,65.0,Anesthesiology
30,1003000142,O,99214,262.0,Anesthesiology
31,1003000142,O,99215,16.0,Anesthesiology


In [15]:
includedSpecialties = [
#    "Addiction Medicine","All Other Suppliers",
    "Allergy/ Immunology",
#    "Ambulance Service Provider","Ambulatory Surgical Center","Anesthesiology",
#    "Anesthesiology Assistant","Audiologist","Cardiac Surgery",
    "Cardiovascular Disease (Cardiology)",
#    "Centralized Flu","Certified Clinical Nurse Specialist",
#    "Certified Nurse Midwife","Certified Registered Nurse Anesthetist (CRNA)",
    "Chiropractic",
#    "Clinic or Group Practice","Clinical Cardiatric Electrophysiology","Clinical Laboratory",
#    "Colorectal Surgery (Proctology)","Critical Care (Intensivists)","Dentist",
    "Dermatology",
#    "Diagnostic Radiology",
    "Emergency Medicine","Endocrinology","Family Practice",
    "Gastroenterology","General Practice","General Surgery","Geriatric Medicine",
#    "Geriatric Psychiatry","Gynecological Oncology","Hand Surgery","Hematology",
#    "Hematology-Oncology","Hospice and Palliative Care","Hospitalist","Independent Diagnostic Testing Facility (IDTF)",
    "Infectious Disease","Internal Medicine","Interventional Cardiology" 
#    ,"Interventional Pain Management",
#    "Interventional Radiology","Licensed Clinical Social Worker","Mammography Center","Mass Immunizer Roster Biller",
#    "Maxillofacial Surgery","Medical Oncology",
    "Nephrology","Neurology",
#    "Neuropsychiatry","Neurosurgery","Nuclear Medicine",
    "Nurse Practitioner","Obstetrics & Gynecology",
#    "Occupational Therapist in Private Practice","Ophthalmology","Optometry","Oral Surgery (Dentists only)",
#    "Orthopedic Surgery","Osteopathic Manipulative Medicine","Otolaryngology","Pain Management",
#    "Pathology","Pediatric Medicine","Peripheral Vascular Disease","Physical Medicine and Rehabilitation",
#    "Physical Therapist in Private Practice","Physician Assistant","Plastic and Reconstructive Surgery",
#    "Podiatry","Portable X-Ray Supplier","Preventive Medicine","Psychiatry","Psychologist, Clinical",
#    "Public Health or Welfare Agency","Pulmonary Disease","Radiation Oncology","Radiation Therapy Center",
#    "Registered Dietitian or Nutrition Professional",
    "Rheumatology"
#    ,
#    "Sleep Medicine","Slide Preparation Facility",
#    "Speech Language Pathologist","Sports Medicine","Surgical Oncology","Thoracic Surgery",
#    "Undefined Physician type","Unknown Supplier/Provider Specialty","Urology","Vascular Surgery"
]

df_med_filtered = df_med_filtered[df_med_filtered.PROVIDER_TYPE.isin(includedSpecialties)]
df_med_filtered.head()

,NPI,PLACE_OF_SERVICE,HCPCS_CODE,LINE_SRVC_CNT,PROVIDER_TYPE
51,1003000423,O,99203,16.0,Obstetrics & Gynecology
52,1003000423,O,99213,27.0,Obstetrics & Gynecology
56,1003000480,F,99202,21.0,General Surgery
57,1003000480,F,99205,11.0,General Surgery
58,1003000480,F,99212,38.0,General Surgery


This cell will take some time to complete.

In [18]:
#df_med_filtered = df_med_filtered.apply(calculateRates, axis=1)
df_med_filtered = apply_by_multiprocessing(df_med_filtered, calculateRates, axis=1, workers=4)

In [19]:
df_medicare_summed = df_med_filtered.groupby('NPI').agg({ 'LINE_SRVC_CNT':sum, 'CURRENT_RATE':sum, 'PROPOSED_RATE':sum, 'PRIMARYCARE_GCODE':sum,
                              'PRIMARYCARE_GCODE_PLUS5':sum, 'PRIMARYCARE_GCODE_PLUS8':sum, 'PRIMARYCARE_GCODE_PLUS10':sum, 'PROLONGED_VISIT_GCODE':sum, 'SPECIALTY_GCODE':sum, 'MIN_SAVED':sum})

In [20]:
df_medicare_agg = pd.read_pickle(medicare_agg_saved)
df_merged = pd.merge(df_medicare_summed, df_medicare_agg, on='NPI', how='left')
df_merged.head()

,NPI,LINE_SRVC_CNT,CURRENT_RATE,PROPOSED_RATE,PRIMARYCARE_GCODE,PRIMARYCARE_GCODE_PLUS5,PRIMARYCARE_GCODE_PLUS8,PRIMARYCARE_GCODE_PLUS10,PROLONGED_VISIT_GCODE,SPECIALTY_GCODE,MIN_SAVED,NPPES_ENTITY_CODE,BENEFICIARY_AVERAGE_RISK_SCORE,NPPES_PROVIDER_STATE,TOTAL_UNIQUE_BENES,PROVIDER_TYPE
0,1003000423,43.0,3758.0,2719.0,0.0,0.0,0.0,0.0,0.0,243.0,68.8,I,0.9540,OH,82,Obstetrics & Gynecology
1,1003000480,94.0,7403.0,10086.0,0.0,0.0,0.0,0.0,515.9,0.0,56.0,I,1.6955,CO,113,General Surgery
2,1003000522,1034.0,98646.0,94005.0,5090.0,10180.0,13234.0,15270.0,770.5,0.0,1569.6,I,1.2619,FL,417,Family Practice
3,1003000530,759.0,80252.0,70587.0,3795.0,7590.0,9867.0,11385.0,1809.0,0.0,1214.4,I,1.0124,PA,334,Internal Medicine
4,1003000902,684.0,56788.0,60972.0,3255.0,6510.0,8463.0,9765.0,0.0,0.0,1072.0,I,1.2416,KY,258,Family Practice


This cell will take some time to complete.

In [22]:
df_merged = apply_by_multiprocessing(df_merged, calculateProposedRates, axis=1, workers=4)

In [23]:
df_merged.head()

,NPI,LINE_SRVC_CNT,CURRENT_RATE,PROPOSED_RATE,PRIMARYCARE_GCODE,PRIMARYCARE_GCODE_PLUS5,PRIMARYCARE_GCODE_PLUS8,PRIMARYCARE_GCODE_PLUS10,PROLONGED_VISIT_GCODE,SPECIALTY_GCODE,...,DIFFERENCE_WITH_GCODE,DIFFERENCE_WITH_GCODE_PLUS5,DIFFERENCE_WITH_GCODE_PLUS8,DIFFERENCE_WITH_GCODE_PLUS10,DIFFERENCE_WITH_GCODE_PER_BENE,PROPOSED_RATE_WITH_GCODE_PER_BENE,DIFFERENCE_WITH_GCODE_PLUS5_PER_BENE,DIFFERENCE_WITH_GCODE_PLUS8_PER_BENE,DIFFERENCE_WITH_GCODE_PLUS10_PER_BENE,DOLLARS_SAVED
0,1003000423,43.0,3758.0,2719.0,0.0,0.0,0.0,0.0,0.0,243.0,...,-1039.0,-1039.0,-1039.0,-1039.0,-12.670732,33.158537,-12.670732,-12.670732,-12.670732,0.00
1,1003000480,94.0,7403.0,10086.0,0.0,0.0,0.0,0.0,515.9,0.0,...,2683.0,2683.0,2683.0,2683.0,23.743363,89.256637,23.743363,23.743363,23.743363,0.00
2,1003000522,1034.0,98646.0,94005.0,5090.0,10180.0,13234.0,15270.0,770.5,0.0,...,449.0,5539.0,8593.0,10629.0,1.076739,237.637890,13.282974,20.606715,25.489209,2432.88
3,1003000530,759.0,80252.0,70587.0,3795.0,7590.0,9867.0,11385.0,1809.0,0.0,...,-5870.0,-2075.0,202.0,1720.0,-17.574850,222.700599,-6.212575,0.604790,5.149701,0.00
4,1003000902,684.0,56788.0,60972.0,3255.0,6510.0,8463.0,9765.0,0.0,0.0,...,7439.0,10694.0,12647.0,13949.0,28.833333,248.941860,41.449612,49.019380,54.065891,1661.60


Calculate some columns for model analysis

In [24]:
df_merged['DIFFERENCE_WITH_GCODE_PLUS_SAVED'] = df_merged['DIFFERENCE_WITH_GCODE']+df_merged['DOLLARS_SAVED']
df_merged['DIFFERENCE_WITH_GCODE_PLUS_PROLONGED'] = df_merged['DIFFERENCE_WITH_GCODE']+df_merged['PROLONGED_VISIT_GCODE']
df_merged['PROPOSED_RATE_FULL'] = df_merged['PROPOSED_RATE'] + df_merged['PRIMARYCARE_GCODE'] + df_merged['SPECIALTY_GCODE']+ df_merged['PROLONGED_VISIT_GCODE'] + df_merged['DOLLARS_SAVED']
df_merged['BENEFICIARY_AVERAGE_RISK_SCORE_BINNED'] = pd.cut(df_merged['BENEFICIARY_AVERAGE_RISK_SCORE'], 3, labels=['low', 'medium', 'high'])

In [25]:
df_merged.to_pickle(medicare_payment_model_saved)

In [26]:
df_merged.columns.values

array(['NPI', 'LINE_SRVC_CNT', 'CURRENT_RATE', 'PROPOSED_RATE',
       'PRIMARYCARE_GCODE', 'PRIMARYCARE_GCODE_PLUS5',
       'PRIMARYCARE_GCODE_PLUS8', 'PRIMARYCARE_GCODE_PLUS10',
       'PROLONGED_VISIT_GCODE', 'SPECIALTY_GCODE', 'MIN_SAVED',
       'NPPES_ENTITY_CODE', 'BENEFICIARY_AVERAGE_RISK_SCORE',
       'NPPES_PROVIDER_STATE', 'TOTAL_UNIQUE_BENES', 'PROVIDER_TYPE',
       'PROPOSED_RATE_WITH_GCODE', 'PROPOSED_RATE_WITH_GCODE_PLUS5',
       'PROPOSED_RATE_WITH_GCODE_PLUS8',
       'PROPOSED_RATE_WITH_GCODE_PLUS10', 'DIFFERENCE',
       'DIFFERENCE_WITH_GCODE', 'DIFFERENCE_WITH_GCODE_PLUS5',
       'DIFFERENCE_WITH_GCODE_PLUS8', 'DIFFERENCE_WITH_GCODE_PLUS10',
       'DIFFERENCE_WITH_GCODE_PER_BENE',
       'PROPOSED_RATE_WITH_GCODE_PER_BENE',
       'DIFFERENCE_WITH_GCODE_PLUS5_PER_BENE',
       'DIFFERENCE_WITH_GCODE_PLUS8_PER_BENE',
       'DIFFERENCE_WITH_GCODE_PLUS10_PER_BENE', 'DOLLARS_SAVED',
       'DIFFERENCE_WITH_GCODE_PLUS_SAVED',
       'DIFFERENCE_WITH_GCODE_PLUS_PR